목표 : 비상장 외감기업 결측치 채우기
=====================
- inf는 max로 대체, -inf는 min으로 대체
- 정규성 검사
- 정규성 -> fillna(mean 평균)
- 정규성X -> fillna(median 중앙값)

import modules

In [54]:
import pandas as pd
import numpy as np

#워닝 메시지
import warnings
warnings.filterwarnings('ignore')

import os
current_directory = os.getcwd()
print("현재 디렉토리:", current_directory)

from scipy.stats import shapiro, anderson, kstest, norm

환경설정

In [2]:
# pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

pd.set_option('display.max_columns', None)
# pd.reset_option('display.max_columns')

pd.set_option('display.max_seq_items', None)

pd.set_option('float_format', '{:.3f}'.format) # 소수점 3개까지 출력

load file

In [4]:
unlisted = pd.read_csv("../datasets/Train_Test/unlisted_train_data.csv")
# 영업년수와 상장년수가 있더라

In [5]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [6]:
len(unlisted)

15138

# describe

In [7]:
unlisted

,회사명,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
0,(주)동남코리아,70118,2022,-18.740,-39.580,-3.410,-27.330,26.090,17.940,22.200,14.910,8.600,-9.020,8.880,3.180,81.830,8.120,15.900,50.950,-2.620,-2.620,-20.070,2.460,0.076,9.948,9.950,-15.220,11.030,13.480,-0.200,8331.620,10.480,29.130,84.140,60.550,69.180,0.125,13.080,5.180,0.350,0.310,0.130,0.140,3.590,30.000,0.414,-1.063,-0.394,76.854,-1.323,1,0,0,0,0.000
1,피에스케이홀딩스(주),20666,2019,130.010,22131.400,10.600,36.690,1385.060,1230.540,8.880,2.110,0.050,24.950,114.300,26.890,91.850,11.820,34.870,36.100,11.380,8.160,27.430,-10.840,-0.105,7.934,6.000,26.260,0.090,3.850,-5.590,10939.850,11.850,334.640,41.020,87.990,81.170,0.235,12.060,1.430,7.920,0.970,0.300,0.400,1.200,38.000,0.289,2.414,0.503,-1.195,-3.927,0,1,0,0,0.000
2,금용해양산업주식회사,63939,2022,121.510,368.960,29.460,46.110,159.900,159.900,32.010,31.960,7.990,14.500,48.050,38.720,75.750,50.580,33.630,47.860,42.540,42.300,60.790,0.510,0.056,19.518,19.520,108.580,-5.870,14.020,-6.550,10122.950,44.910,81.610,70.280,68.090,97.140,0.639,8.690,2.230,1.120,0.700,0.700,1.010,2.230,14.000,0.049,1.080,0.999,-2.679,-1.741,0,1,0,0,0.000
3,주식회사대일,53349,2014,-2.390,-3.210,-1.850,-2.010,87.410,23.480,341.770,259.400,57.610,-7.390,1.420,51.330,22.640,24.050,5.060,5.290,9.260,9.230,9.860,3.640,-0.168,4.012,4.010,-3.050,12.670,16.740,13.680,2957.410,16.310,37.170,17.760,29.790,71.880,0.919,9.350,6.370,2.230,0.960,0.940,2.000,1.760,22.000,0.102,2.030,1.585,0.070,-0.142,1,0,0,0,1.000
4,주식회사삼호프레스,83444,2018,-1.480,-2.030,-0.910,-2.540,71.900,59.910,160.760,62.430,45.020,-6.730,0.120,17.210,38.350,8.830,4.510,9.030,2.970,1.630,4.630,0.110,0.148,-3.881,-3.880,17.000,-7.320,-4.730,-0.270,1084.780,9.050,20.560,25.230,35.800,60.410,0.359,3.290,2.700,0.810,0.500,0.350,0.420,2.250,22.000,0.237,6.115,0.447,-0.385,0.462,0,0,0,1,0.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15133,농업회사법인한만두식품주식회사,82850,2022,6.720,8.850,5.280,5.950,95.860,63.560,367.770,204.360,59.700,-1.810,25.700,41.880,21.380,14.330,2.970,3.220,4.500,4.360,4.760,1.640,0.064,6.283,6.280,20.560,-2.070,6.990,-1.990,3759.790,16.650,29.810,18.740,17.170,58.460,0.888,7.680,3.440,1.550,0.920,0.920,1.510,2.320,13.000,0.172,4.615,1.414,-0.155,0.420,0,0,1,0,0.000
15134,금성냉장주식회사,29681,2014,-0.530,-1.020,-0.440,-0.520,28.600,13.240,484.900,416.960,43.490,-50.900,0.100,20.390,17.100,-20.450,-3.930,-4.560,-3.260,-3.570,-4.270,1.600,-0.075,-4.291,-4.290,0.740,-5.500,-16.420,-5.030,725.240,7.510,9.810,8.790,-51.880,41.560,0.854,68.170,10.460,1.090,0.860,0.830,1.040,4.200,24.000,0.083,-0.938,0.937,-1.821,-2.265,0,1,0,0,1.000
15135,(주)노비아레텍,70975,2022,-7.670,-11.400,-5.630,-4.340,100.960,38.140,276.910,264.210,49.430,0.670,1.150,70.770,26.530,2.860,0.780,0.580,1.540,1.520,1.120,1.510,-0.053,9.701,9.700,11.770,4.990,38.430,-0.200,1397.660,7.420,47.950,5.710,10.080,51.480,1.299,7.650,5.730,8.380,1.360,1.360,4.550,1.940,12.000,0.069,7.126,1.497,3.183,-0.746,1,0,0,0,1.000
15136,새한에프앤비(주),13761,2014,-9.190,-10.580,-6.360,-17.210,45.060,45.060,224.730,202.640,60.150,-34.290,0.920,28.120,30.800,-14.520,-9.610,-13.480,-5.090,-5.090,-14.260,0.000,0.083,-6.617,-6.620,8.800,-11.520,39.820,-10.570,3102.700,20.900,113.540,56.550,-23.830,19.250,0.370,0.000,1.370,1.900,0.710,0.360,0.480,1.370,31.000,1.143,-7.013,1.058,-0.725,2.522,0,0,0,1,1.000


In [8]:
unlisted.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15137.000,15138.000,14908.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,67773.011,2019.874,19.314,724.234,3.241,-22.741,431.245,358.475,1378.936,983.416,43.176,0.167,98.637,45.603,38.894,6.064,2.195,-3.021,2.665,2.244,-18.975,9.249,-3.649,inf,6.597,216.233,302.142,228.063,192381.296,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.938,700.636,4.246,7686.666,1.119,0.996,22.988,2.830,21.322,0.503,9.588,0.895,117.567,-5.947,0.218,0.368,0.191,0.223,0.361
std,23619.347,2.797,1261.272,31970.320,37.792,539.239,5347.663,5224.565,35986.322,30885.126,211.937,297.974,2442.606,28.781,26.118,196.933,34.633,436.198,12.496,12.451,261.639,113.199,486.055,NaN,55.243,20598.824,11247.279,9908.536,21029765.042,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,1.663,46090.572,7.518,263256.703,1.590,1.404,375.217,5.156,11.151,2.602,364.674,32.023,30082.690,678.697,0.413,0.482,0.393,0.416,0.480
min,10037.000,2014.000,-86467.650,-323703.660,-3388.620,-21110.050,0.070,0.070,0.000,0.000,0.000,-36261.200,-0.200,0.000,0.000,-4261.270,-949.400,-20068.670,-309.770,-309.800,-18444.330,-2399.950,-59524.677,-100.000,-99.890,-99.900,-100.000,-99.910,-100.000,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-9823.523,-3383.817,-1368867.400,-35585.135,0.000,0.000,0.000,0.000,0.000
25%,52329.000,2018.000,-4.480,-5.120,-2.880,-4.890,56.050,31.620,60.845,36.860,17.360,-19.517,1.590,21.870,18.270,-4.720,-2.518,-5.040,-1.108,-1.530,-3.377,0.140,-0.166,-4.447,-5.480,-14.060,-5.207,-12.510,-5.630,937.122,4.870,9.130,7.960,-0.310,22.102,0.162,3.990,1.460,0.650,0.340,0.280,0.490,1.060,13.000,0.086,-0.136,0.453,-2.527,-1.760,0.000,0.000,0.000,0.000,0.000
50%,70853.000,2022.000,2.920,1.510,1.860,2.360,108.715,72.700,163.855,102.565,42.040,2.925,8.605,43.180,35.145,3.610,1.410,1.600,2.540,2.190,2.830,1.360,-0.020,0.523,1.820,3.990,-0.110,3.610,-1.040,2281.575,11.060,28.820,16.220,11.390,55.870,0.652,7.150,2.880,2.060,0.820,0.730,1.460,1.990,20.000,0.162,2.100,0.956,-0.802,-0.770,0.000,0.000,0.000,0.000,0.000
75%,87046.750,2022.000,14.328,17.380,7.387,10.777,197.823,150.323,373.438,239.108,61.450,26.938,32.528,67.630,57.267,14.308,6.390,7.090,6.690,6.297,9.408,4.510,0.062,11.191,12.805,25.038,12.480,21.797,8.002,4883.170,19.218,97.688,33.188,35.943,74.770,1.231,13.370,4.940,7.050,1.430,1.300,3.757,3.340,27.000,0.393,4.274,1.352,-0.063,0.216,0.000,1.000,0.000,0.000,1.000
max,101635.000,2022.000,113999.840,3194164.340,1274.750,21579.540,447760.110,447760.110,3385006.150,2905990.690,25856.360,99.690,275285.960,100.000,99.980,12490.440,2735.330,17430.100,110.740,109.710,2136.190,10135.130,5597.416,inf,5081.220,2531488.340,1154781.210,900275.000,2580453089.000,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,100.540,5386048.240,344.060,19823298.290,106.800,92.050,32610.280,344.060,110.000,183.355,31473.399,743.445,3001862.889,32320.540,1.000,1.000,1.000,1.000,1.000


In [9]:
unlisted.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15138 entries, 0 to 15137
Data columns (total 55 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   회사명                15138 non-null  object 
 1   거래소코드              15138 non-null  int64  
 2   회계년도               15138 non-null  int64  
 3   CASH FLOW 대 부채비율   15138 non-null  float64
 4   CASH FLOW 대 차입금비율  15138 non-null  float64
 5   CASH FLOW 대 총자본비율  15138 non-null  float64
 6   CASH FLOW 대 매출액비율  15138 non-null  float64
 7   유동비율               15138 non-null  float64
 8   당좌비율               15138 non-null  float64
 9   부채비율               15138 non-null  float64
 10  유동부채비율             15138 non-null  float64
 11  차입금의존도             15138 non-null  float64
 12  순운전자본비율            15138 non-null  float64
 13  현금비율               15138 non-null  float64
 14  유동자산구성비율           15138 non-null  float64
 15  자기자본구성비율           15138 non-null  float64
 16  자기자본순이익률           151

In [10]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [37]:
unlisted.isna().sum().sort_values(ascending=False)
# 결측치는 뒤에서 정규성 검사를 하고 대체한다.

영업재무현금비율             230
재무레버리지                 1
회사명                    0
경영자본회전률                0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
비유동자산회전률               0
총자본회전률                 0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


In [13]:
unlisted.drop(['부실판단', '도입기', '성장기', '성숙기', '쇠퇴기'], axis=1).isin([0]).sum().sort_values(ascending=False)

자본분배율                1598
이윤분배율                1388
총자산회전율               1354
총자산증가율               1353
차입금의존도               1273
CASH FLOW 대 차입금비율    1268
매출채권회전률              1181
자기자본순이익률              755
자기자본구성비율              601
부채비율                  599
유동부채비율                599
유형자산증가율               348
설비투자효율                216
유형자산회전율               189
매출액증가율                145
현금비율                   90
금융비용부담률                83
총자본회전률                 54
비유동자산증가율               46
경영자본회전률                37
비유동자산회전률               28
영업년수                   16
유동자산회전률                14
쭈피처                    13
총자본사업이익률                9
총자본증가율                  8
총자본영업이익률                8
매출액영업이익률                8
CASH FLOW 대 총자본비율       8
CASH FLOW 대 매출액비율       8
경영자본순이익률                7
CASH FLOW 대 부채비율        5
유동자산증가율                 4
매출액순이익률                 4
총자본투자효율                 4
당좌자산회전률                 2
CROE                    2
부가가치율                   2
순운전자본비율     

In [36]:
unlisted.isin([np.inf]).sum().sort_values(ascending=False)
# inf가 하나 있다. 뒤에서 max 값으로 대체한다.

총자산증가율               1
회사명                  0
매출액증가율               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
유동자산증가율              0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율   

In [15]:
unlisted.isin([-np.inf]).sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

# 범주 분할 1현금흐름 / 2안정성 / 2수익성 / 3유동성 / 4성장성 / 5생산성 / 6활동성 / 7시장지표 및 기타

In [16]:
# 비상장 all data는 unlisted
# 비상장 label들

labels = unlisted[
    [
        '거래소코드',
        '회계년도',
        '부실판단',
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기'
    ]
]

# 비상장 피처
features = unlisted.drop(
    labels=[
        '회사명',
        '거래소코드',
        '회계년도',        
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'        
    ],
    axis=1,    
)

# 1현금흐름 4개
cash_ft1 = unlisted[
    [
        'CASH FLOW 대 부채비율',
        'CASH FLOW 대 차입금비율',
        'CASH FLOW 대 총자본비율',
        'CASH FLOW 대 매출액비율',
        '부실판단'
    ]
]

# 2안정성 9개
stability_ft1 = unlisted[
    [
        '유동비율',
        '당좌비율',
        '부채비율',
        '유동부채비율',
        '차입금의존도',
        '순운전자본비율',
        '현금비율',
        '유동자산구성비율',
        '자기자본구성비율',
        '부실판단'
    ]
]

# 3수익성 8개
profitability_ft1 = unlisted[
    [
        '자기자본순이익률',
        '경영자본순이익률',
        '매출액순이익률',
        '총자본사업이익률',
        '총자본영업이익률',
        '매출액영업이익률',
        '금융비용부담률',
        'CROE',
        '부실판단'
    ]
]

# 4성장성 6개
growth_ft1 = unlisted[
    [
        '총자산증가율',
        '총자본증가율',
        '유동자산증가율',
        '비유동자산증가율',
        '매출액증가율',
        '유형자산증가율',
        '부실판단'
    ]
]

# 5생산성 6개
productivity_ft1 = unlisted[
    [
        '부가가치(백만원)',
        '총자본투자효율',
        '설비투자효율',
        '부가가치율',
        '이윤분배율',
        '자본분배율',
        '부실판단'
    ]
]

# 6활동성 8개
activity_ft1 = unlisted[
    [
        '총자산회전율',
        '매출채권회전률',
        '당좌자산회전률',
        '유형자산회전율',
        '경영자본회전률',
        '총자본회전률',
        '비유동자산회전률',
        '유동자산회전률',
        '부실판단'
    ]
]

# 7기타
etc_ft1 = unlisted[
    [
        '영업년수',
        '쭈피처',
        '영업레버리지',
        '재무레버리지',
        '영업투자현금비율',
        '영업재무현금비율', 
        '도입기',
        '성숙기',
        '성장기',
        '쇠퇴기',
        '부실판단'
    ]
]

In [17]:
unlisted.columns

Index(['회사명', '거래소코드', '회계년도', 'CASH FLOW 대 부채비율', 'CASH FLOW 대 차입금비율',
       'CASH FLOW 대 총자본비율', 'CASH FLOW 대 매출액비율', '유동비율', '당좌비율', '부채비율',
       '유동부채비율', '차입금의존도', '순운전자본비율', '현금비율', '유동자산구성비율', '자기자본구성비율',
       '자기자본순이익률', '경영자본순이익률', '매출액순이익률', '총자본사업이익률', '총자본영업이익률', '매출액영업이익률',
       '금융비용부담률', 'CROE', '총자산증가율', '총자본증가율', '유동자산증가율', '비유동자산증가율', '매출액증가율',
       '유형자산증가율', '부가가치(백만원)', '총자본투자효율', '설비투자효율', '부가가치율', '이윤분배율', '자본분배율',
       '총자산회전율', '매출채권회전률', '당좌자산회전률', '유형자산회전율', '경영자본회전률', '총자본회전률',
       '비유동자산회전률', '유동자산회전률', '영업년수', '쭈피처', '영업레버리지', '재무레버리지', '영업투자현금비율',
       '영업재무현금비율', '도입기', '성숙기', '성장기', '쇠퇴기', '부실판단'],
      dtype='object')

In [18]:
# 1현금흐름 4가지 cash
cash_ft1.describe()

,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000
mean,19.314,724.234,3.241,-22.741,0.361
std,1261.272,31970.320,37.792,539.239,0.480
min,-86467.650,-323703.660,-3388.620,-21110.050,0.000
25%,-4.480,-5.120,-2.880,-4.890,0.000
50%,2.920,1.510,1.860,2.360,0.000
75%,14.328,17.380,7.387,10.777,1.000
max,113999.840,3194164.340,1274.750,21579.540,1.000


In [19]:
# 2안정성 9가지 stability
stability_ft1.describe()

,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,431.245,358.475,1378.936,983.416,43.176,0.167,98.637,45.603,38.894,0.361
std,5347.663,5224.565,35986.322,30885.126,211.937,297.974,2442.606,28.781,26.118,0.480
min,0.070,0.070,0.000,0.000,0.000,-36261.200,-0.200,0.000,0.000,0.000
25%,56.050,31.620,60.845,36.860,17.360,-19.517,1.590,21.870,18.270,0.000
50%,108.715,72.700,163.855,102.565,42.040,2.925,8.605,43.180,35.145,0.000
75%,197.823,150.323,373.438,239.108,61.450,26.938,32.528,67.630,57.267,1.000
max,447760.110,447760.110,3385006.150,2905990.690,25856.360,99.690,275285.960,100.000,99.980,1.000


In [20]:
# 3수익성 8가지 profitability
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


In [21]:
# 4성장성 6개 growth
growth_ft1.describe()

,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,inf,6.597,216.233,302.142,228.063,192381.296,0.361
std,NaN,55.243,20598.824,11247.279,9908.536,21029765.042,0.480
min,-100.000,-99.890,-99.900,-100.000,-99.910,-100.000,0.000
25%,-4.447,-5.480,-14.060,-5.207,-12.510,-5.630,0.000
50%,0.523,1.820,3.990,-0.110,3.610,-1.040,0.000
75%,11.191,12.805,25.038,12.480,21.797,8.002,1.000
max,inf,5081.220,2531488.340,1154781.210,900275.000,2580453089.000,1.000


In [22]:
# 5생산성 6가지 productivity
productivity_ft1.describe()

,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,6674.974,30.528,538176.590,46.170,-95.676,8.411,0.361
std,352176.022,1669.910,34787344.705,747.415,2660.033,1074.584,0.480
min,-68167.780,-246.020,-563419210.000,-13763.270,-208392.960,-94627.640,0.000
25%,937.122,4.870,9.130,7.960,-0.310,22.102,0.000
50%,2281.575,11.060,28.820,16.220,11.390,55.870,0.000
75%,4883.170,19.218,97.688,33.188,35.943,74.770,1.000
max,43327598.520,205392.780,4053778655.000,70947.820,8226.360,126.190,1.000


In [23]:
# 6활동성 8가지 activity
activity_ft1.describe()

,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,부실판단
count,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,0.938,700.636,4.246,7686.666,1.119,0.996,22.988,2.830,0.361
std,1.663,46090.572,7.518,263256.703,1.590,1.404,375.217,5.156,0.480
min,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000
25%,0.162,3.990,1.460,0.650,0.340,0.280,0.490,1.060,0.000
50%,0.652,7.150,2.880,2.060,0.820,0.730,1.460,1.990,0.000
75%,1.231,13.370,4.940,7.050,1.430,1.300,3.757,3.340,1.000
max,100.540,5386048.240,344.060,19823298.290,106.800,92.050,32610.280,344.060,1.000


In [24]:
# 7기타 etc
etc_ft1.describe()

,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,15138.000,15138.000,15138.000,15137.000,15138.000,14908.000,15138.000,15138.000,15138.000,15138.000,15138.000
mean,21.322,0.503,9.588,0.895,117.567,-5.947,0.218,0.368,0.191,0.223,0.361
std,11.151,2.602,364.674,32.023,30082.690,678.697,0.413,0.482,0.393,0.416,0.480
min,0.000,0.000,-9823.523,-3383.817,-1368867.400,-35585.135,0.000,0.000,0.000,0.000,0.000
25%,13.000,0.086,-0.136,0.453,-2.527,-1.760,0.000,0.000,0.000,0.000,0.000
50%,20.000,0.162,2.100,0.956,-0.802,-0.770,0.000,0.000,0.000,0.000,0.000
75%,27.000,0.393,4.274,1.352,-0.063,0.216,0.000,1.000,0.000,0.000,1.000
max,110.000,183.355,31473.399,743.445,3001862.889,32320.540,1.000,1.000,1.000,1.000,1.000


# 기업수명주기와 부실별

In [25]:
# 도입정상
도입정상 = unlisted[(unlisted['도입기'] == 1) & (unlisted['부실판단'] == 0)]

In [26]:
도입정상.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000,1290.000
mean,72404.669,2021.247,19.001,21.451,-0.474,8.587,373.650,226.199,745.043,489.127,46.145,3.822,31.190,49.582,35.128,0.800,-0.475,-11.885,1.749,1.433,-8.642,6.620,0.002,19.249,20.658,43.147,287.847,581.686,13974.820,3211.665,12.716,184065.787,17.372,-99.612,11.023,1.111,37.913,4.509,6706.710,1.406,1.260,18.362,2.707,20.396,0.335,5.519,1.301,265.168,-11.120,1.000,0.000,0.000,0.000,0.000
std,23062.266,1.793,460.373,713.767,12.779,463.055,3468.137,2278.656,3769.845,2440.955,22.168,32.575,180.017,26.441,20.738,202.250,18.060,127.192,8.950,8.941,108.112,47.739,4.795,145.054,145.089,427.752,5591.712,20072.840,394090.258,4680.866,26.945,6013981.702,119.146,1718.136,399.344,2.727,268.453,6.213,187524.733,3.468,3.076,267.404,4.126,10.363,1.027,41.280,19.495,6398.209,239.314,0.000,0.000,0.000,0.000,0.000
min,10495.000,2014.000,-404.230,-4429.660,-89.480,-3241.290,0.310,0.240,0.000,0.000,0.000,-95.330,0.000,0.180,0.000,-4078.270,-414.710,-2911.330,-55.090,-55.110,-3141.370,-370.510,-43.798,-86.291,-86.290,-97.110,-99.570,-99.870,-99.930,-13273.590,-136.860,-17320027.900,-1467.540,-58129.420,-10103.230,0.000,0.000,0.010,0.000,0.000,0.000,0.010,0.010,0.000,0.004,-179.876,-561.165,-2.202,-8428.705,1.000,0.000,0.000,0.000,0.000
25%,58692.750,2022.000,-6.115,-8.945,-3.725,-4.995,67.127,33.912,106.347,69.510,30.782,-16.260,1.385,29.375,20.352,-7.510,-3.050,-5.022,-1.020,-1.298,-2.343,0.400,-0.265,0.000,-0.318,-8.275,-1.613,-18.562,-2.993,971.745,4.453,9.315,6.123,-5.938,13.922,0.259,4.230,1.722,0.992,0.472,0.400,0.810,1.220,12.250,0.078,-0.063,0.620,0.314,-1.027,1.000,0.000,0.000,0.000,0.000
50%,77770.000,2022.000,-1.310,-1.725,-0.770,-0.880,107.235,62.800,210.650,138.925,47.525,2.765,6.425,48.610,31.920,2.490,0.875,0.870,2.420,2.100,2.060,1.225,-0.072,6.629,8.475,11.220,3.960,2.065,0.025,2229.500,9.435,26.665,11.720,7.755,47.920,0.761,7.200,3.255,2.670,0.990,0.880,1.915,2.080,19.000,0.148,2.584,1.035,0.955,-0.632,1.000,0.000,0.000,0.000,0.000
75%,91112.250,2022.000,2.940,4.008,1.818,1.858,162.553,111.735,384.765,281.993,62.720,22.805,17.700,68.670,48.045,10.032,3.578,3.110,5.092,4.835,5.575,3.238,0.004,23.004,24.945,34.715,23.580,19.883,22.027,4298.990,14.935,73.940,20.630,25.593,66.090,1.336,12.410,5.360,7.540,1.627,1.500,4.330,3.260,26.000,0.320,5.569,1.629,3.624,-0.281,1.000,0.000,0.000,0.000,0.000
max,101375.000,2022.000,15418.200,23164.090,182.960,13622.540,83360.650,70031.060,92834.240,47766.780,100.310,98.140,3676.260,100.000,99.280,5412.140,195.930,1929.130,54.560,52.000,79.100,1436.010,127.589,5081.217,5081.220,13329.590,185726.400,720869.030,13454040.000,61774.370,363.860,211718878.000,3460.750,173.170,113.560,80.578,4723.230,109.160,6402030.360,106.800,92.050,9261.970,109.160,77.000,32.414,1188.538,158.829,188183.733,4.057,1.000,0.000,0.000,0.000,0.000


In [27]:
# 도입정상 결측치확인
도입정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [28]:
# 도입부실
도입부실 = unlisted[(unlisted['도입기'] == 1) & (unlisted['부실판단'] == 1)]

In [29]:
도입부실.describe()

,거래소코드,회계년도,CASH FLOW 대 부채비율,CASH FLOW 대 차입금비율,CASH FLOW 대 총자본비율,CASH FLOW 대 매출액비율,유동비율,당좌비율,부채비율,유동부채비율,차입금의존도,순운전자본비율,현금비율,유동자산구성비율,자기자본구성비율,자기자본순이익률,경영자본순이익률,매출액순이익률,총자본사업이익률,총자본영업이익률,매출액영업이익률,금융비용부담률,CROE,총자산증가율,총자본증가율,유동자산증가율,비유동자산증가율,매출액증가율,유형자산증가율,부가가치(백만원),총자본투자효율,설비투자효율,부가가치율,이윤분배율,자본분배율,총자산회전율,매출채권회전률,당좌자산회전률,유형자산회전율,경영자본회전률,총자본회전률,비유동자산회전률,유동자산회전률,영업년수,쭈피처,영업레버리지,재무레버리지,영업투자현금비율,영업재무현금비율,도입기,성숙기,성장기,쇠퇴기,부실판단
count,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000,2011.000
mean,66440.453,2018.022,-13.501,-45.717,-7.943,-145.555,174.501,114.405,1782.400,987.750,57.986,-7.272,26.946,44.203,25.592,-23.491,-6.209,-91.275,-3.287,-3.582,-72.591,24.739,-0.105,18.587,20.850,76.534,684.738,477.322,1293273.080,1677.787,6.806,313656.965,-8.237,-262.359,-52.438,0.619,111.538,4.582,15484.456,0.808,0.704,48.779,2.664,19.652,1.039,0.455,0.927,2293.079,-2.145,1.000,0.000,0.000,0.000,1.000
std,23423.882,2.801,34.745,664.677,11.515,952.495,577.908,488.786,10247.473,5301.155,29.419,42.042,194.438,31.956,21.957,111.892,44.858,696.871,16.554,16.595,581.840,144.357,11.224,49.004,53.023,486.590,11895.583,7321.955,57542879.468,5145.591,51.641,11023800.224,458.604,4402.666,2143.884,0.951,1585.439,11.016,427838.092,1.148,0.947,518.211,6.280,11.337,5.774,263.726,10.751,73619.123,46.534,0.000,0.000,0.000,0.000,0.000
min,10062.000,2014.000,-798.400,-28558.800,-176.290,-21110.050,0.070,0.070,0.000,0.000,0.000,-364.500,0.000,0.010,0.000,-1853.830,-949.400,-20068.670,-309.770,-309.800,-18444.330,-1554.580,-206.365,-72.941,-72.940,-98.710,-97.310,-99.830,-100.000,-52877.960,-120.130,-89493757.170,-13763.270,-184890.760,-94627.640,0.000,0.000,0.010,0.000,0.000,0.000,0.000,0.000,0.000,0.000,-9823.523,-354.852,-159.772,-1993.452,1.000,0.000,0.000,0.000,1.000
25%,52262.000,2015.000,-14.440,-20.185,-10.215,-44.980,37.265,16.330,107.695,56.015,39.500,-28.545,0.480,15.345,6.885,-30.980,-8.550,-36.245,-4.965,-5.120,-20.105,1.140,-0.365,-0.182,-0.905,-8.165,-2.550,-14.780,-4.055,194.910,0.865,1.215,3.270,-76.020,0.000,0.053,2.835,1.110,0.200,0.120,0.100,0.160,0.640,11.000,0.110,-1.312,0.310,0.444,-0.947,1.000,0.000,0.000,0.000,1.000
50%,70048.000,2018.000,-6.270,-8.460,-4.570,-10.830,87.510,43.540,260.980,155.290,58.380,-4.790,2.350,40.810,21.580,-3.090,-2.450,-6.050,-0.460,-0.770,-2.030,3.610,-0.051,4.818,6.570,11.080,1.740,3.190,-0.410,1108.170,4.810,11.180,12.130,0.000,35.670,0.356,5.900,2.560,1.360,0.530,0.460,0.990,1.490,17.000,0.237,0.282,0.804,1.555,-0.636,1.000,0.000,0.000,0.000,1.000
75%,85193.000,2021.000,-2.220,-2.660,-1.660,-3.285,142.670,89.465,713.725,439.545,76.400,17.175,9.270,70.370,38.605,2.700,1.050,1.675,2.520,2.175,4.610,15.590,0.061,19.984,24.230,41.110,21.010,38.255,13.740,2534.190,10.530,54.090,29.350,10.995,68.290,0.862,12.670,4.800,6.540,1.090,0.950,3.425,2.685,25.000,0.680,3.142,1.270,6.451,-0.296,1.000,0.000,0.000,0.000,1.000
max,101548.000,2022.000,190.180,288.930,32.100,2636.780,11325.470,11325.470,234003.100,142937.490,405.580,97.420,5412.970,100.000,98.370,1012.280,1217.950,3177.900,50.010,50.000,99.560,5273.750,269.597,1026.841,1026.840,8846.890,418389.230,241171.310,2580453089.000,58251.770,2179.890,402503633.330,4384.980,2781.640,118.000,23.652,59829.560,246.350,17320295.670,24.120,20.200,17302.990,151.080,79.000,183.355,4658.556,114.148,3001862.889,469.354,1.000,0.000,0.000,0.000,1.000


In [30]:
# 도입부실 결측치확인
도입부실.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [31]:
# 성장정상
성장정상 = unlisted[(unlisted['성장기'] == 1) & (unlisted['부실판단'] == 0)]

In [32]:
# 결측치확인
성장정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [34]:
# 성장부실
성장부실 = unlisted[(unlisted['성장기'] == 1) & (unlisted['부실판단'] == 1)]

In [35]:
# 결측치 확인
성장부실.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [38]:
# inf 확인
성장부실.isin([np.inf]).sum().sort_values(ascending=False)

총자산증가율               1
회사명                  0
매출액증가율               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
총자본회전률               0
비유동자산회전률             0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
유동자산증가율              0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
현금비율                 0
유동자산구성비율   

In [39]:
# inf 값 대체
inf_col = 성장부실.isin([np.inf]).sum().sort_values(ascending=False).index[0]
max_value = 성장부실[inf_col].replace(np.inf, np.nan).max()
성장부실[inf_col] = 성장부실[inf_col].replace(np.inf, max_value)

In [43]:
성장부실[inf_col].isin([np.inf]).sum()
# .sort_values(ascending=False)

0

In [44]:
# 성숙정상
성숙정상 = unlisted[(unlisted['성숙기'] == 1) & (unlisted['부실판단'] == 0)]

In [45]:
# 성숙정상 결측치 확인
성숙정상.isna().sum().sort_values(ascending=False)

회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
재무레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [46]:
# 성숙부실
성숙부실 = unlisted[(unlisted['성숙기'] == 1) & (unlisted['부실판단'] == 1)]

In [47]:
# 성숙부실 결측치 확인
성숙부실.isna().sum().sort_values(ascending=False)

재무레버리지               1
회사명                  0
총자본회전률               0
부가가치(백만원)            0
총자본투자효율              0
설비투자효율               0
부가가치율                0
이윤분배율                0
자본분배율                0
총자산회전율               0
매출채권회전률              0
당좌자산회전률              0
유형자산회전율              0
경영자본회전률              0
비유동자산회전률             0
매출액증가율               0
유동자산회전률              0
영업년수                 0
쭈피처                  0
영업레버리지               0
영업투자현금비율             0
영업재무현금비율             0
도입기                  0
성숙기                  0
성장기                  0
쇠퇴기                  0
유형자산증가율              0
비유동자산증가율             0
거래소코드                0
현금비율                 0
회계년도                 0
CASH FLOW 대 부채비율     0
CASH FLOW 대 차입금비율    0
CASH FLOW 대 총자본비율    0
CASH FLOW 대 매출액비율    0
유동비율                 0
당좌비율                 0
부채비율                 0
유동부채비율               0
차입금의존도               0
순운전자본비율              0
유동자산구성비율             0
유동자산증가율              0
자기자본구성비율   

In [48]:
# 정규성 확인 하기
na_col0 = 성숙부실.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 성숙부실[na_col0].dropna()

In [50]:
# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.3310622572898865, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [52]:
# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 185.93136271858157
유의수준 15.0%: 임계값 = 0.573
유의수준 10.0%: 임계값 = 0.653
유의수준 5.0%: 임계값 = 0.784
유의수준 2.5%: 임계값 = 0.914
유의수준 1.0%: 임계값 = 1.087
데이터는 정규 분포를 따르지 않습니다.


In [55]:
# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.14866872064673958, p-value: 4.8506857072186544e-18
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [56]:
# fillna median
성숙부실[na_col0].fillna(data0.median(), inplace=True)

In [57]:
# 쇠퇴정상
쇠퇴정상 = unlisted[(unlisted['쇠퇴기'] == 1) & (unlisted['부실판단'] == 0)]

In [58]:
# 결측치 확인
쇠퇴정상.isna().sum().sort_values(ascending=False)

영업재무현금비율             124
회사명                    0
총자본회전률                 0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
경영자본회전률                0
비유동자산회전률               0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업레버리지                 0
재무레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


In [59]:
# 정규성 확인하기
na_col0 = 쇠퇴정상.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴정상[na_col0].dropna()

In [60]:
# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.01838862895965576, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [61]:
# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 503.70699486728176
유의수준 15.0%: 임계값 = 0.574
유의수준 10.0%: 임계값 = 0.654
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.915
유의수준 1.0%: 임계값 = 1.089
데이터는 정규 분포를 따르지 않습니다.


In [62]:
# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.1679789483977252, p-value: 6.862637710098656e-34
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [63]:
# 쇠퇴정상 fillna median
쇠퇴정상[na_col0].fillna(data0.median(), inplace=True)

In [64]:
# 쇠퇴부실
쇠퇴부실 = unlisted[(unlisted['쇠퇴기'] == 1) & (unlisted['부실판단'] == 1)]

In [66]:
# 결측치 확인
쇠퇴부실.isna().sum().sort_values(ascending=False)

영업재무현금비율             106
회사명                    0
총자본회전률                 0
부가가치(백만원)              0
총자본투자효율                0
설비투자효율                 0
부가가치율                  0
이윤분배율                  0
자본분배율                  0
총자산회전율                 0
매출채권회전률                0
당좌자산회전률                0
유형자산회전율                0
경영자본회전률                0
비유동자산회전률               0
매출액증가율                 0
유동자산회전률                0
영업년수                   0
쭈피처                    0
영업레버리지                 0
재무레버리지                 0
영업투자현금비율               0
도입기                    0
성숙기                    0
성장기                    0
쇠퇴기                    0
유형자산증가율                0
비유동자산증가율               0
거래소코드                  0
현금비율                   0
회계년도                   0
CASH FLOW 대 부채비율       0
CASH FLOW 대 차입금비율      0
CASH FLOW 대 총자본비율      0
CASH FLOW 대 매출액비율      0
유동비율                   0
당좌비율                   0
부채비율                   0
유동부채비율                 0
차입금의존도                 0


In [67]:
# 정규성 확인하기
na_col0 = 쇠퇴부실.isna().sum().sort_values(ascending=False).index[0]

# 테스트를 위한 샘플 데이터 생성
data0 = 쇠퇴부실[na_col0].dropna()

In [68]:
# 샤피로-윌크 검정 수행
statistic, p_value = shapiro(data0)

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.01935398578643799, p-value: 0.0
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [69]:
# 안더슨-달링 검정 수행
result = anderson(data0)

# 검정 통계량과 임계값 출력
print(f"검정 통계량: {result.statistic}")

# 임계값과 유의수준에 따른 결과 출력
for i in range(len(result.critical_values)):
    sl, cv = result.significance_level[i], result.critical_values[i]
    print(f"유의수준 {sl}%: 임계값 = {cv}")

# 검정 통계량이 임계값보다 작으면 정규 분포를 따른다고 판단
if result.statistic < result.critical_values[2]:
    print("데이터는 정규 분포를 따릅니다.")
else:
    print("데이터는 정규 분포를 따르지 않습니다.")

검정 통계량: 666.3560402047547
유의수준 15.0%: 임계값 = 0.575
유의수준 10.0%: 임계값 = 0.655
유의수준 5.0%: 임계값 = 0.785
유의수준 2.5%: 임계값 = 0.916
유의수준 1.0%: 임계값 = 1.09
데이터는 정규 분포를 따르지 않습니다.


In [70]:
# Kolmogorov-Smirnov 검정 수행
statistic, p_value = kstest(data0, 'norm')

# 검정 통계량과 p-value 출력
print(f"검정 통계량: {statistic}, p-value: {p_value}")

# p-value를 기준으로 귀무가설을 기각 여부 판단
alpha = 0.05
if p_value > alpha:
    print("데이터는 정규 분포를 따릅니다 (귀무가설을 기각하지 않음)")
else:
    print("데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)")

검정 통계량: 0.19980988647451559, p-value: 4.839024161896977e-63
데이터는 정규 분포를 따르지 않습니다 (귀무가설을 기각)


In [71]:
# fillna(median)
쇠퇴부실[na_col0].fillna(data0.median(), inplace=True)

# to_csv

In [76]:
unlisted.loc[도입부실.index] = 도입부실
unlisted.loc[도입정상.index] = 도입정상
unlisted.loc[성장부실.index] = 성장부실
unlisted.loc[성장정상.index] = 성장정상
unlisted.loc[성숙부실.index] = 성숙부실
unlisted.loc[성숙정상.index] = 성숙정상
unlisted.loc[쇠퇴부실.index] = 쇠퇴부실
unlisted.loc[쇠퇴정상.index] = 쇠퇴정상

In [78]:
unlisted.to_csv('../datasets/Train_Test/unlisted_filled_train_data.csv', index=False)